In [3]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [4]:
subj = subject.Subject & {'subject_uuid': UUID('122205e8-0a31-41ce-9632-30e7857a959d')}
# get trial counts and session length to date
session_info = (behavior.TrialSet * acquisition.Session & subj).proj(
    'n_trials', session_date='DATE(session_start_time)',
    session_duration='TIMESTAMPDIFF(MINUTE, session_start_time, session_end_time)').fetch(as_dict=True)
session_info = pd.DataFrame(session_info)
session

In [5]:
trial_counts = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['n_trials'].tolist(),
    mode = 'markers+lines',
    marker = dict(
        size = 6,
        color = 'black'),
    name = 'trial counts',
    yaxis = 'y1'
)
session_length = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['session_duration'].tolist(),
    mode = 'markers+lines',
    marker = dict(
        size = 6,
        color = 'red'),
    name = 'session duration',
    yaxis = 'y2'
)

data = [trial_counts, session_length]

layout = go.Layout(
    yaxis=dict(
        title='Trial counts',
    ),
    yaxis2=dict(
        title='Session duration (mins)',
        overlaying = 'y',
        color = 'red',
        side='right'
    ),
    xaxis={'title':'Date'}, 
    width=500,
    height=400,
    title='Trial counts and session duration',
    legend=dict(
        x=0, 
        y=1.1,
        orientation='v'),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [6]:
f = open("trial_counts_session_duration.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()